In [2]:
import pandas as pd
import numpy as np

# Алгоритм проверки гипотез
__Задача__ - разработать процедуру проверки подбрасываемой монеты на честность. 

__Алгоритм:__
1. Формулируем гипотезы:
    - $H_{0}$ - монета честная, Орел выпадает в 50% случаев; 
    - $H_{1}$ - монета не честная. Орел выпадает не в 50% случаев;
2. Собираем доказательства, которые смогут опровергнуть $H_{0}$ - проводим эксперимент.
3. Оцениваем собранные доказательства через вопросы - _"Насколько сильно полученные данные противоречат $H_{0}$? Достаточно ли силы у полученных данных, чтобы опровергнуть $H_{}0$?"_
4. Формируем процедуру проверки:
    - Так как проверка на честность не имеет направления (не важно, что выпадает чаще Орел/Решка), то для упрощения решения можно смотреть только __модуль__ значения отклонения от нормы. 
    - Так как проверка будет иметь бинарный ответ (монета честная или нет), то мы можем использовать метрики бинарных тестов (`TPR`, `FPR`, `MDE`) для оценки качества разрабатываемой процедуры по соответствующим метрикам (`TPR`, `FPR`, `MDE`).
    - Раз у нас будут метрики, то перед началом разработки самой процедуры стоит зафиксировать желаемые метрики.  
5. Проверяем процедуру проверки монеты используя возможности метода Монте-Карло.


## Coin check

Желаемые метрики:
- `MDE`: 10% - какую разницу хотим детектировать;
- `Power[1 - beta]` or `TPR`: 80% - в скольки % случаев наша процедура должна корректно определять нечестность монеты;
- `Signigicance[alpha]` or `FPR`: 5% - в скольки % случаев наша процедура может ошибаться и допускать ошибку 1-го рода;

In [98]:
def coin_check(sample_size=None, threshold=None, mde=None, n=None):
    """
    Функция, которая выполняет процедуру проверки честности монеты в соответствии с заданными метриками.
    :param sample_size: Размер выборки (количество бросков монетки в одном эксперименте).
    :param threshold: Порог отклонения от нормы.
    :param mde: Желаемый уровень детектирования честности/не честности монетки.
    :param n: Количество экспериментов.
    :return: Метрики проведенных экспериментов. 
    """
    
    # TPR 
    tpr = []
    for i in range(n):
        np.random.seed(i)
        tpr.append(np.random.binomial(sample_size, 0.5 + mde))

    # FPR 
    fpr = []
    for i in range(n):
        np.random.seed(i)
        fpr.append(np.random.binomial(sample_size, 0.5))

    # Metrics
    df = pd.DataFrame({"tpr": tpr, "fpr": fpr})
    df["tpr_difference"] = abs(sample_size * 0.5 - df["tpr"])
    df["fpr_difference"] = abs(sample_size * 0.5 - df["fpr"])
    tpr_result = (df["tpr_difference"] >= sample_size * threshold).mean()
    fpr_result = (df["fpr_difference"] >= sample_size * threshold).mean()

    # Results
    print(f"Sample size: {sample_size}, Threshold: {threshold:.2%}|{sample_size * threshold:.2f}, TPR: {tpr_result:.2%}, FPR: {fpr_result:.2%}, MDE: {mde:.2%}")


In [103]:
for i in [10, 100, 150, 200, 250]:
    coin_check(sample_size=i, threshold=0.05, mde=0.1, n=100_000)

Sample size: 10, Threshold: 5.00%|0.50, TPR: 79.71%, FPR: 75.48%, MDE: 10.00%
Sample size: 100, Threshold: 5.00%|5.00, TPR: 87.03%, FPR: 36.97%, MDE: 10.00%
Sample size: 150, Threshold: 5.00%|7.50, TPR: 89.37%, FPR: 22.05%, MDE: 10.00%
Sample size: 200, Threshold: 5.00%|10.00, TPR: 93.49%, FPR: 17.99%, MDE: 10.00%
Sample size: 250, Threshold: 5.00%|12.50, TPR: 94.60%, FPR: 11.51%, MDE: 10.00%


In [100]:
for i in [10, 100, 150, 200, 250]:
    coin_check(sample_size=i, threshold=0.06, mde=0.1, n=100_000)

Sample size: 10, Threshold: 6.00%|0.60, TPR: 79.71%, FPR: 75.48%, MDE: 10.00%
Sample size: 100, Threshold: 6.00%|6.00, TPR: 82.11%, FPR: 27.21%, MDE: 10.00%
Sample size: 150, Threshold: 6.00%|9.00, TPR: 86.10%, FPR: 16.42%, MDE: 10.00%
Sample size: 200, Threshold: 6.00%|12.00, TPR: 89.07%, FPR: 10.46%, MDE: 10.00%
Sample size: 250, Threshold: 6.00%|15.00, TPR: 91.25%, FPR: 6.69%, MDE: 10.00%


In [101]:
for i in [10, 100, 150, 200, 250]:
    coin_check(sample_size=i, threshold=0.07, mde=0.1, n=100_000)

Sample size: 10, Threshold: 7.00%|0.70, TPR: 79.71%, FPR: 75.48%, MDE: 10.00%
Sample size: 100, Threshold: 7.00%|7.00, TPR: 69.64%, FPR: 13.29%, MDE: 10.00%
Sample size: 150, Threshold: 7.00%|10.50, TPR: 77.30%, FPR: 8.55%, MDE: 10.00%
Sample size: 200, Threshold: 7.00%|14.00, TPR: 78.68%, FPR: 4.01%, MDE: 10.00%
Sample size: 250, Threshold: 7.00%|17.50, TPR: 83.44%, FPR: 2.69%, MDE: 10.00%


Перебрав несколько вариантов `sample_size` и `threshold` находим наилучший вариант параметров для заданных нами метрик.
- Хотим получить: `MDE` - 10%, `TPR` - 80%, `FPR` - 5%
- При `Sample size` - 250 и `Threshold` - 7% получили: `MDE` - 10%, `TPR` - 83.44%, `FPR` - 2.69%   

## Satisfaction check
__Задача__ - Мы знаем на основании исторических данных, что удовлетворенность покупателей нашего магазина 80%. Но, в последнее время есть подозрения, что удовлетворенность покупателей снизилась. Необходимо понять - так ли это на самом деле. 

__Алгоритм:__
1. Формулируем гипотезы:
    - $H_{0}$ - удовлетворенность клиентов 80%; 
    - $H_{1}$ - удовлетворенность клиентов меньше 80%;
2. Желаемые метрики:
    - `MDE`: 10%;
    - `Power[1 - beta]` or `TPR`: 80%;
    - `Signigicance[alpha]` or `FPR`: 5%;


In [111]:
def satisfaction_check(sample_size=None, threshold=None, mde=None, n=None, baseline=None):
    """
    Функция, которая выполняет процедуру проверки уменьшения удовлетворенности покупателей.
    :param sample_size: Размер выборки (какому количеству покупателей звоним и спрашиваем их удовлетворенность?).
    :param threshold: Порог отклонения от нормы.
    :param mde: Желаемый уровень детектирования изменения удовлетворенности
    :param n: Количество экспериментов.
    :return: Метрики проведенных экспериментов. 
    """

    # TPR 
    tpr = []
    for i in range(n):
        np.random.seed(i)
        tpr.append(np.random.binomial(sample_size, baseline - mde))

    # FPR 
    fpr = []
    for i in range(n):
        np.random.seed(i)
        fpr.append(np.random.binomial(sample_size, baseline))

    # Metrics
    df = pd.DataFrame({"tpr": tpr, "fpr": fpr})
    df["tpr_difference"] = df["tpr"] - sample_size * baseline
    df["fpr_difference"] = df["fpr"] - sample_size * baseline
    
    tpr_result = (df["tpr_difference"] <= -1 * sample_size * threshold).mean()
    fpr_result = (df["fpr_difference"] <= -1 * sample_size * threshold).mean()

    # Results
    print(f"Sample size: {sample_size}, Threshold: {threshold:.2%}|{sample_size * threshold:.2f}, TPR: {tpr_result:.2%}, FPR: {fpr_result:.2%}, MDE: {mde:.2%}")


In [118]:
for i in [10, 100, 110, 120, 130]:
    satisfaction_check(sample_size=i, threshold=0.05, mde=0.1, n=100_000, baseline=0.8)

Sample size: 10, Threshold: 5.00%|0.50, TPR: 61.83%, FPR: 32.27%, MDE: 10.00%
Sample size: 100, Threshold: 5.00%|5.00, TPR: 88.58%, FPR: 13.11%, MDE: 10.00%
Sample size: 110, Threshold: 5.00%|5.50, TPR: 87.32%, FPR: 9.77%, MDE: 10.00%
Sample size: 120, Threshold: 5.00%|6.00, TPR: 90.38%, FPR: 10.70%, MDE: 10.00%
Sample size: 130, Threshold: 5.00%|6.50, TPR: 89.43%, FPR: 8.03%, MDE: 10.00%


In [119]:
for i in [10, 100, 110, 120, 130]:
    satisfaction_check(sample_size=i, threshold=0.06, mde=0.1, n=100_000, baseline=0.8)

Sample size: 10, Threshold: 6.00%|0.60, TPR: 61.83%, FPR: 32.27%, MDE: 10.00%
Sample size: 100, Threshold: 6.00%|6.00, TPR: 83.60%, FPR: 8.78%, MDE: 10.00%
Sample size: 110, Threshold: 6.00%|6.60, TPR: 82.25%, FPR: 6.48%, MDE: 10.00%
Sample size: 120, Threshold: 6.00%|7.20, TPR: 81.23%, FPR: 4.76%, MDE: 10.00%
Sample size: 130, Threshold: 6.00%|7.80, TPR: 85.28%, FPR: 5.41%, MDE: 10.00%


In [120]:
for i in [10, 100, 110, 120, 130]:
    satisfaction_check(sample_size=i, threshold=0.07, mde=0.1, n=100_000, baseline=0.8)

Sample size: 10, Threshold: 7.00%|0.70, TPR: 61.83%, FPR: 32.27%, MDE: 10.00%
Sample size: 100, Threshold: 7.00%|7.00, TPR: 70.40%, FPR: 3.48%, MDE: 10.00%
Sample size: 110, Threshold: 7.00%|7.70, TPR: 76.21%, FPR: 4.09%, MDE: 10.00%
Sample size: 120, Threshold: 7.00%|8.40, TPR: 75.30%, FPR: 3.03%, MDE: 10.00%
Sample size: 130, Threshold: 7.00%|9.10, TPR: 74.61%, FPR: 2.24%, MDE: 10.00%


Итого. Для того чтобы принять $H_{1}$ нам необходимо обзвонить 120 покупателей и если отклонение от нормы (норма = 80% от 120) будет 7 человек и более (threshold = 6% от 120), то мы сможем принять данную гипотезу. И при этом у нас будут выдержанны желаемые (заданные) метрики.